In [21]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.metrics import SCORERS
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
#from lightgbm import LGBMRegressor
#import xgboost as xgb


#next: try boosting algorithms, and gridsearch aiming to improve SVC or even better reduces forests overfitting
#download 2022 squads to make predictions

In [3]:
#loading data and retaining only significant columns
df=pd.read_csv('NBA_Pts.csv', sep=';', decimal=',')
df=df[df['Team2']!='nok'][['Season_endyear', 'Player', 'Pos', 'Age', 'G', 'PTS', 'Team2']]
df.head(5)

,Season_endyear,Player,Pos,Age,G,PTS,Team2
0,2010,Arron Afflalo,SG,24,82,8.829268,DEN
1,2010,Alexis Ajinça,C,21,6,1.666667,CHO
2,2010,LaMarcus Aldridge,PF,24,78,17.858974,POR
3,2010,Joe Alexander,SF,23,8,0.500000,CHI
4,2010,Malik Allen,PF,31,51,2.058824,DEN


In [4]:
#searching for serious injuries through game tresholds

df_games=df.copy()
for year in range(5):
    df_games['G-'+str(year+1)]=(df_games.sort_values(by=['Season_endyear'], ascending=True).groupby(['Player'])['G'].shift(year+1))

df_games['pts_injury_adjusted']=np.where(df_games['G']/df_games['G-1']<0.5, 1, 0)
#df_games['std']=df_games[['G-1','G-2','G-3','G-4','G-5']].std(axis=1)
#df_games['hist']=((df_games['G']-df_games['mean'])/df_games['mean'])
#df_games['low_tres']=df_games['mean']-df_games['std']
#df_games[df_games['Player']=='Stephen Curry']
#df_games[df_games['Player']=='Kawhi Leonard']
#df_games[df_games['hist']<1]['hist'].hist(bins=50)
df_games

,Season_endyear,Player,Pos,Age,G,PTS,Team2,G-1,G-2,G-3,G-4,G-5,pts_injury_adjusted
0,2010,Arron Afflalo,SG,24,82,8.829268,DEN,74.0,NaN,NaN,NaN,NaN,0
1,2010,Alexis Ajinça,C,21,6,1.666667,CHO,31.0,NaN,NaN,NaN,NaN,1
2,2010,LaMarcus Aldridge,PF,24,78,17.858974,POR,81.0,NaN,NaN,NaN,NaN,0
3,2010,Joe Alexander,SF,23,8,0.500000,CHI,59.0,NaN,NaN,NaN,NaN,1
4,2010,Malik Allen,PF,31,51,2.058824,DEN,49.0,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8067,2011,Dorell Wright,SF,25,82,16.390244,GSW,72.0,6.0,NaN,NaN,NaN,0
8068,2011,Julian Wright,SF,23,52,3.615385,TOR,68.0,54.0,NaN,NaN,NaN,0
8069,2011,Nick Young,SG,25,64,17.421875,WAS,74.0,82.0,NaN,NaN,NaN,0
8070,2011,Sam Young,SF,25,78,7.333333,MEM,80.0,NaN,NaN,NaN,NaN,0


In [17]:
#creating last 5 years data


for year in range(5):
    df['PTS_year-'+str(year+1)]=(df.sort_values(by=['Season_endyear'], ascending=True).groupby(['Player'])['PTS'].shift(year+1))
#test - adjusting for injuries   
df['PTS_y']=df['PTS']
df2=df.merge(df_games[['Season_endyear', 'Player', 'pts_injury_adjusted']], on=['Season_endyear', 'Player'])
df2['mean']=df2[['PTS_year-1', 'PTS_year-2', 'PTS_year-3', 'PTS_year-4', 'PTS_year-5']].mean(axis=1)
df2['PTS']=np.where(df2['pts_injury_adjusted']==0, df2['PTS'], df2['mean'])
for year in range(5):
    df2['PTS_year-'+str(year+1)]=(df2.sort_values(by=['Season_endyear'], ascending=True).groupby(['Player'])['PTS'].shift(year+1))

df=df2.drop(['pts_injury_adjusted', 'mean'], axis=1)
#df2[df2['Player']=='Stephen Curry']
df[df['Player']=='Stephen Curry']

,Season_endyear,Player,Pos,Age,G,PTS,Team2,PTS_year-1,PTS_year-2,PTS_year-3,PTS_year-4,PTS_year-5,PTS_y
99,2010,Stephen Curry,PG,21,80,17.487500,GSW,NaN,NaN,NaN,NaN,NaN,17.487500
1004,2021,Stephen Curry,PG,32,63,32.000000,GSW,26.563418,27.300000,26.400000,25.303797,30.063291,32.000000
1549,2020,Stephen Curry,PG,31,5,26.563418,GSW,27.300000,26.400000,25.303797,30.063291,23.750000,20.800000
2081,2019,Stephen Curry,PG,30,69,27.300000,GSW,26.400000,25.303797,30.063291,23.750000,24.012821,27.300000
2607,2018,Stephen Curry,PG,29,51,26.400000,GSW,25.303797,30.063291,23.750000,24.012821,22.897436,26.400000
3125,2017,Stephen Curry,PG,28,79,25.303797,GSW,30.063291,23.750000,24.012821,22.897436,18.020777,25.303797
3618,2016,Stephen Curry,PG,27,79,30.063291,GSW,23.750000,24.012821,22.897436,18.020777,18.554054,30.063291
4101,2015,Stephen Curry,PG,26,80,23.750000,GSW,24.012821,22.897436,18.020777,18.554054,17.487500,23.750000
4592,2014,Stephen Curry,PG,25,78,24.012821,GSW,22.897436,18.020777,18.554054,17.487500,NaN,24.012821
5062,2013,Stephen Curry,PG,24,78,22.897436,GSW,18.020777,18.554054,17.487500,NaN,NaN,22.897436


In [33]:
#creating the yearly current team past total ppg

df_team=df.groupby(['Season_endyear', 'Team2'])[['PTS_year-1','PTS_year-2','PTS_year-3','PTS_year-4','PTS_year-5']].sum().reset_index()


#creating the yearly team-pos total ppg
df_pos=df.groupby(['Season_endyear', 'Team2', 'Pos'])[['PTS_year-1','PTS_year-2','PTS_year-3','PTS_year-4','PTS_year-5']].sum().reset_index()

#creating player bins
df_bins=df[['Season_endyear', 'Team2', 'PTS_y']].copy()
for pts in [5, 10, 15, 20, 25, 30]:
    df_bins['bin_'+str(pts)]=np.where(df_bins['PTS_y']>=pts,1,0)
df_bins=df_bins.groupby(['Season_endyear', 'Team2']).sum().reset_index().drop('PTS_y', axis=1)

for year in range(5):
    for bins in [5, 10, 15, 20, 25, 30]:
        df_bins['bin_'+str(bins)+'-'+str(year+1)]=(df_bins.sort_values(by=['Season_endyear'], ascending=True).groupby(['Team2'])['bin_'+str(bins)].shift(year+1))


#merging in the final dataframe
df_stats=df.merge(df_team, on=['Season_endyear', 'Team2'], suffixes=('','_team'))
df_stats=df_stats.merge(df_pos, on=['Season_endyear', 'Team2', 'Pos'], suffixes=('','_pos'))
df_stats=df_stats.merge(df_bins, on=['Season_endyear', 'Team2'])
df_stats.head(5)

#drop season prior to 2013 for lack of data
df_stats=df_stats[df_stats['Season_endyear']>2013]

#replacing NA with 0. Usually it will mean a new player, sometimes can account for long-term injuries. But let's see how it goes

df_stats=df_stats.fillna(0)

In [34]:
#separating train and test sets

X=df_stats[df_stats['Season_endyear']<2021].drop(['Season_endyear', 'Player', 'G', 'Team2', 'Pos', 'PTS', 'PTS_y'], axis=1)
X_oot=df_stats[df_stats['Season_endyear']==2021].drop(['Season_endyear', 'Player', 'G', 'Team2','Pos', 'PTS', 'PTS_y'], axis=1)
y=df_stats[df_stats['Season_endyear']<2021]['PTS_y']
y_oot=df_stats[df_stats['Season_endyear']==2021]['PTS_y']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=8)



In [35]:
#training and evaluating the models
lr=LinearRegression()
sv=SVR()
rf=RandomForestRegressor()
#lgbm=lightgbm.LGBMRegressor()

r2=[]
mse=[]
models=[]
dataset=[]

for model in (lr, sv, rf):
    model.fit(X_train, y_train)
    r2.append(r2_score(y_train, model.predict(X_train)))
    r2.append(r2_score(y_test, model.predict(X_test)))
    r2.append(r2_score(y_oot, model.predict(X_oot)))
    mse.append(mean_squared_error(y_train, model.predict(X_train)))
    mse.append(mean_squared_error(y_test, model.predict(X_test)))
    mse.append(mean_squared_error(y_oot, model.predict(X_oot)))
    dataset.append(['train', 'test', 'oot'])
    models.append([str(model),str(model),str(model)])


In [36]:
#evaluating results
r2

[0.7163147552416176,
 0.7029681506766505,
 0.7368669119068478,
 0.4805432824022463,
 0.47274462740656165,
 0.4579671117605978,
 0.9598564809319929,
 0.7218103863608571,
 0.7645864899354577]

In [37]:
#evaluating results
mse

[9.936504587162055,
 10.620923196914815,
 11.041823988193284,
 18.194756874431015,
 18.852991119411936,
 22.745264729431323,
 1.4060874463309319,
 9.947184207254237,
 9.878630473318609]

In [42]:
#checking 2021 predictions vs. actual

df_check=df_stats[df_stats['Season_endyear']==2021][['Player', 'PTS_y']].copy()
df_check['prediction']=rf.predict(X_oot)
df_check['error']=df_check['PTS_y']-df_check['prediction']
#df_check.sort_values(by=['prediction'], ascending=False).head(10)
df_check.sort_values(by=['error'], ascending=False).head(10)

,Player,PTS_y,prediction,error
915,Elijah Bryant,16.0,3.249804,12.750196
1277,Jerami Grant,22.3,10.897817,11.402183
1262,Anthony Edwards,19.3,8.424180,10.875820
1355,Khyri Thomas,16.4,5.845751,10.554249
1191,Michael Porter Jr.,19.0,9.761608,9.238392
1067,Gary Trent Jr.,15.3,7.100896,8.199104
1353,Victor Oladipo,19.8,11.962641,7.837359
1185,Immanuel Quickley,11.4,3.596068,7.803932
1260,Malik Beasley,19.6,11.836706,7.763294
1362,Christian Wood,21.0,13.289780,7.710220


In [30]:
df_check[(df_check['Player'] == 'Stephen Curry') | (df_check['Player'] == 'Bradley Beal') | (df_check['Player'] == 'Russell Westbrook')]


,Player,PTS_y,prediction
1038,Anthony Davis,21.8,25.882077
1119,Russell Westbrook,22.2,27.316454
1230,Stephen Curry,32.0,26.791358


In [39]:
df_stats[(df_stats['Season_endyear'] == 2021) & ((df_stats['Player'] == 'Stephen Curry') | (df_stats['Player'] == 'Anthony Davis') | (df_stats['Player'] == 'Russell Westbrook'))]

,Season_endyear,Player,Pos,Age,G,PTS,Team2,PTS_year-1,PTS_year-2,PTS_year-3,...,bin_15-4,bin_20-4,bin_25-4,bin_30-4,bin_5-5,bin_10-5,bin_15-5,bin_20-5,bin_25-5,bin_30-5
1038,2021,Anthony Davis,PF,27,36,21.8,LAL,26.100000,25.9,28.1,...,2.0,0.0,0.0,0.0,10.0,5.0,3.0,0.0,0.0,0.0
1119,2021,Russell Westbrook,PG,32,65,22.2,WAS,27.200000,22.9,25.4,...,2.0,2.0,0.0,0.0,11.0,4.0,2.0,0.0,0.0,0.0
1230,2021,Stephen Curry,PG,32,63,32.0,GSW,26.563418,27.3,26.4,...,3.0,3.0,2.0,0.0,10.0,4.0,2.0,2.0,1.0,1.0
